In [10]:
%matplotlib inline 
# plots graphs within the notebook
%config InlineBackend.figure_format='svg' # not sure what this does, may be default images to svg format
from __future__ import division
from IPython.display import Image

from IPython.core.display import HTML
def header(text):
    raw_html = '<h4>' + str(text) + '</h4>'
    return raw_html

def box(text):
    raw_html = '<div style="border:1px dotted black;padding:2em;">'+str(text)+'</div>'
    return HTML(raw_html)

def nobox(text):
    raw_html = '<p>'+str(text)+'</p>'
    return HTML(raw_html)

def addContent(raw_html):
    global htmlContent
    htmlContent += raw_html

### Library for thermodynamic properties

Defines thermodynamic properties of air and water at 1 atm. Air properties are tabled between $-150\text{$^\circ$C}$ and $400\text{$^\circ$C}$, water between $274\text{K}$ and $373\text{K}$, Argon between $100\text{K}$ and $700\text{K}$ and Krypton between $150\text{K}$ and $750\text{K}$
<p class='alert alert-danger'>
<b>Input must be in Kelvin</b>
</p>
Use the scipy functions <FONT FACE="courier" style="color:blue">C2K </FONT> and <FONT FACE="courier" style="color:blue">F2K </FONT> to convert temperatures to Kelvin. Thermodynamic properties are linearly interpolated from the two nearest temperature states.

In [11]:
%%file thermodynamics.py
""" Object name: Fluid"""
import numpy as np
import scipy
import scipy.optimize
from scipy.constants.constants import C2K
from scipy.constants.constants import K2C
from scipy.constants.constants import F2K
from scipy.constants.constants import K2F
import scipy.constants as sc

def interpolate_table(target,index,xquantity,yquantity):
    return yquantity[index] + \
                (yquantity[index+1]-yquantity[index])* \
                (target-xquantity[index])/(xquantity[index+1]-xquantity[index])
        
class Fluid(object):
    """ Compute thermodynamics properties of air between -150 C and 400 C, 
        water between 274K and 373K, argon between 100 and 700K and
        krypton between 150 and 700 K under 1 atm. Argon, krypton and water were obtained 
        through http://webbook.nist.gov/chemistry/fluid/
        Declare your fluid as fluid=Fluid('name') where
        name is air, water, argon or krypton
        Then run fluid.get_properties(T) where T is your temperature
        Note that the temperature must be in Kelvin
        More fluids to be added in the future"""
    def __init__(self,name):
        self.name = name
        
    def get_properties(self,T_o):
        self.T = T_o
        if self.name == 'water':
            if T_o < 274 or T_o > 373:
                print("Temperature is out of bounds for liquid water")
                return 
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.genfromtxt('Tables/water1atm.csv', delimiter=',', skip_header = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T_o-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'argon':
            if T_o < 100 or T_o > 700:
                print("Temperature is out of bounds for argon")
                return 
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.loadtxt('Tables/Argon1atm.csv', delimiter=',', skiprows = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T_o-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'krypton':
            if T_o < 150 or T_o > 740:
                print("Temperature is out of bounds for krypton")
                return 
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.loadtxt('Tables/Krypton1atm.csv', delimiter=',', skiprows = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T_o-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'air':
            if T_o < C2K(-150.) or T_o > C2K(400.):
                print("Temperature is out of bounds of the table for air")
                return
            Ttab,rhotab,Cptab,ktab,nutab,betatab,Prtab = \
            np.genfromtxt('Tables/air1atm.csv', delimiter=',', skip_header = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Ttab = C2K(Ttab)
            Cptab *= 1e3
            nutab *= 1e-6
            mutab = rhotab*nutab
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            i = 0
            while (Ttab[i] < T_o) and (i<Ntab):
                i += 1
            i -=1
            if (i == Ntab - 1):
                i = Ntab - 2
            
        else:
            print("warning, no table available for", self.name)
            return
        
        self.rho = interpolate_table(T_o,i,Ttab,rhotab)
        self.Cp = interpolate_table(T_o,i,Ttab,Cptab)
        self.mu = interpolate_table(T_o,i,Ttab,mutab)
        self.k = interpolate_table(T_o,i,Ttab,ktab)
        self.nu = interpolate_table(T_o,i,Ttab,nutab)
        self.alpha = interpolate_table(T_o,i,Ttab,alphatab)
        self.Pr = interpolate_table(T_o,i,Ttab,Prtab)
        if (self.name == 'air'):
            self.beta = 1./T_o
        else:
            self.beta = interpolate_table(T_o,i,Ttab,betatab)
        

Overwriting thermodynamics.py


In [12]:
import thermodynamics as thermo
import numpy as np
from scipy.constants.constants import C2K
from scipy.constants.constants import K2C
from scipy.constants.constants import F2K
from scipy.constants.constants import K2F

fluid = thermo.Fluid('argon')
print(fluid.name)
#array = np.genfromtxt('Tables/Argon1atm.csv', delimiter=',', skip_header = 1, unpack=True, dtype=float)
#print(array[0,:])
fluid.get_properties(C2K(5.))
print(fluid.k)

argon
0.01662565


### Library of thermal resistances

In [13]:
%%file HT_thermal_resistance.py
"""Object name: Resistance
   Function name: serial_sum(R,nori,nend), performs serial sum of a resistance object list from nori to nend
   Function name: parallel_sum(R,nori,nend), performs parallel sum of a resistance object list from nori to nend
   """
### definition of thermal resistance ###
from __future__ import division
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')


from IPython.display import display,Image, Latex
import numpy as np
import math
import scipy.constants as sc

import sympy as sym
#from sympy import *

class Resistance(object):
    """ Defines thermal resistances for conduction, convection and radiation heat transfer. 
        First define the object attached with class with the name used in the thermal circuit
        and the units, which can only be 'W', 'W/m' or 'W/m^2'
        Second use self.conduction, self.convection or self.radiation to calculate your 
        resistance. Each mode requires different arguments:
        
        .conduction(geo,k,r_a,r_b,A,r_a_name,r_b_name,A_name,Ta_name,Tb_name), where geo can only be 'plane',
        'cylindrical' or 'spherical', r_a is the first length and the only that matters in case
        of planar conduction, r_b is the second length. r_b must be an input even for planar
        (could be 0.). A is the surface area of the system for plane conduction, or the length of the cylinder
        for cylindrical conduction. Set to 0 if spherical. All arguments ending with _name are
        used to write the flux equations(they are strings preferably LaTeX formatted)
        
        .convection(h,A,h_name,A_name,Ta_name,Tb_name), where h is the convection coefficient (W/m^2K) and A is 
        the surface area. All arguments ending with _name are used to write the flux equations(they are strings 
        preferably LaTeX formatted)
        
        .radiation(eps,T_s,T_sur,A,h_name,A_name,Ta_name,Tb_name), where eps is the permissivity of the material, T_s
        the surface temperature, T_sur the far away surface temperature, A the surface area.
        
        .contact(R,A,R_name,A_name,Ta_name,Tb_name), where R is the contact resistance, typically obtained from a table
        A is the surface area
        
        .display_equation(index) displays the heat flux/rate equations for a given resistance. index is the number of 
        your resistance (you specify)
        
        
        
        """
    def __init__(self,name,units):
        self.name = name
        self.units = units
    def conduction(self,geo,k,ra,rb,A,k_name,ra_name,rb_name,A_name,Ta_name,Tb_name):
        self.geometry = geo
        self.mode = 'conduction'
        self.k_name = k_name
        self.ra_name = ra_name
        self.rb_name = rb_name
        self.surface_name = A_name
        self.surface_scale = A
        self.Ta_name = Ta_name
        self.Tb_name = Tb_name
        if self.geometry == 'plane':
            self.R = ra/(k*A)
        elif self.geometry == 'cylindrical':
            self.R = np.log(rb/ra)/(2.*math.pi*self.surface_scale*k)
        elif self.geometry == 'spherical':
            self.R = (1./ra-1./rb)/(4.*math.pi*k)
        else :
            print("geometry is not plane, cylindrical or spherical, cannot compute")
    def convection(self,h,A,h_name,A_name,Ta_name,Tb_name):
        self.mode = 'convection'
        self.R = 1./(h*A)
        self.surface_scale = A
        self.h = h
        self.h_name = h_name
        self.surface_name = A_name
        self.Ta_name = Ta_name
        self.Tb_name = Tb_name
    def radiation(self,eps,T_s,T_sur,A,h_name,A_name,Ta_name,Tb_name):
        self.R = 1./(eps*sc.sigma*(T_s+T_sur)*(T_s**2+T_sur**2)*A)
        self.mode = 'radiation'
        self.surface_scale = A
        self.h = eps*sc.sigma*(T_s+T_sur)*(T_s**2+T_sur**2)
        self.surface_name = A_name
        self.h_name = h_name
        self.Ta_name = Ta_name
        self.Tb_name = Tb_name
    def contact(self,R,A,R_name,A_name,Ta_name,Tb_name):
        self.R = R/A
        self.mode = 'contact'
        self.R_name = R_name
        self.surface_scale = A
        self.surface_name = A_name
        self.Ta_name = Ta_name
        self.Tb_name = Tb_name
        
    def display_equation(self,index):

        Tasym = sym.symbols(self.Ta_name)
        Tbsym = sym.symbols(self.Tb_name)
        if self.units == 'W':
            Asym = sym.symbols(self.surface_name)
            namesym = "q_"+str(index)
        elif self.units == 'W/m':
            Asym = sym.symbols(self.surface_name)
            namesym = "q'_"+str(index)
        elif self.units == 'W/m^2':
            namesym = "q''_"+str(index)
        else:
            print('units are not properly defined')
        qsym = sym.symbols(namesym)
        Rsym = sym.symbols(self.name[1:-1])
        eq = sym.Eq(qsym,(1/Rsym)*(Tasym-Tbsym))
        if self.mode == 'conduction':
            rasym = sym.symbols(self.ra_name)
            rbsym = sym.symbols(self.rb_name)
            cstsym = sym.symbols(self.k_name)
            if self.geometry == 'plane':
                if self.units != 'W/m^2':
                    eq1 = sym.Eq(qsym,cstsym*Asym/rasym*(Tasym-Tbsym))
                else:
                    eq1 = sym.Eq(qsym,cstsym/rasym*(Tasym-Tbsym))
            elif self.geometry == 'cylindrical':
                if self.units == 'W':
                    eq1 = sym.Eq(qsym,2*sym.pi*cstsym/sym.log(rbsym/rasym)*Asym*(Tasym-Tbsym))
                else:
                    eq1 = sym.Eq(qsym,2*sym.pi*cstsym/sym.log(rbsym/rasym)*(Tasym-Tbsym))
            elif self.geometry == 'spherical':
                eq1 = sym.Eq(qsym,4*sym.pi*cstsym/(1/rasym-1/rbsym)*(Tasym-Tbsym))
                
        elif self.mode == 'convection':
            cstsym = sym.symbols(self.h_name)
            if self.units == 'W/m^2':
                eq1 = sym.Eq(qsym,cstsym*(Tasym-Tbsym))
            else:
                eq1 = sym.Eq(qsym,cstsym*Asym*(Tasym-Tbsym))
        elif self.mode == 'radiation':
            cstsym = sym.symbols(self.h_name)
            if self.units == 'W/m^2':
                eq1 = sym.Eq(qsym,cstsym*(Tasym-Tbsym))
            else:
                eq1 = sym.Eq(qsym,cstsym*Asym*(Tasym-Tbsym))
        elif self.mode == 'contact':
            cstsym = sym.symbols(self.R_name)
            if self.units == 'W/m^2':
                eq1 = sym.Eq(qsym,cstsym*(Tasym-Tbsym))
            else:
                eq1 = sym.Eq(qsym,(Asym/cstsym)*(Tasym-Tbsym))
        
        return display(eq,eq1)
        
### summation of thermal resistance (R is a vector) ###
def serial_sum(R,nori,nend):
    sum = 0.
    for i in range(nori,nend+1):
        sum += R[i].R
    return sum

def parallel_sum(R,nori,nend):
    sum = 0.
    for i in range(nori,nend+1):
        sum += 1./R[i].R
    return 1./sum




Overwriting HT_thermal_resistance.py


In [14]:
import HT_thermal_resistance as res
from HT_thermal_resistance import Resistance,serial_sum,parallel_sum

Rth = []
Rth.append(Resistance('$R_{cond.1}$','W/m^2'))
k = 0.1
ra = 0.008
rb = 0.009
A =0.8*0.5
k_name = 'k_glass'
ra_name = 'r_1'
rb_name = 'r_2'
A_name = 'A'
Ta_name = 'T_{in}'
Tb_name = 'T_{out}'


Rth[0].conduction('plane',k,ra,rb,A,k_name,ra_name,rb_name,A_name,Ta_name,Tb_name)
Rth[0].display_equation(0)
print(Rth[0].surface_name)
Rth.append(Resistance('$R_{conv.1}$','W/m^2'))
h = 10.
h_name = 'h_{c.1}'
Ta_name = 'T_{out}'
Tb_name = 'T_{\infty.out}'
Rth[1].convection(h,A,h_name,A_name,Ta_name,Tb_name)
Rth[1].display_equation(1)
#Rth[1].conduction('plane',10.,1.,0.,1.)
#print(Rth[1].name)
?res

#print(parallel_sum(Rth,0,1))
    
#R_total = serial_sum(Rth[:].R)
#print(R_total)

       T_{in} - T_{out}
q''₀ = ────────────────
          R_{cond.1}   

       k_glass⋅(T_{in} - T_{out})
q''₀ = ──────────────────────────
                   r₁            

A


       -T_{\infty.out} + T_{out}
q''₁ = ─────────────────────────
               R_{conv.1}       

q''₁ = h_{c.1}⋅(-T_{\infty.out} + T_{out})

<h3> Library for extended surfaces</h3>

In [15]:
%%file HT_conduction_extended_surfaces.py
"""Object: ExtSurfaces"""
from __future__ import division
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')


from IPython.display import display,Image, Latex
import numpy as np
import math
import scipy.constants as sc

import sympy as sym
#from sympy import *

class ExtSurfaces(object):
    """ Defines temperature distribution, heat rate for constant cross sectional area fins.
        from Libraries import HT_conduction_extended_surfaces as condext
        
        fin = condext.ExtSurfaces(T_b,T_infty,T_L,k,h,P,Ac,L)
            will calculate fin.m, fin.M which are constants used in flux calculation.
        fin.heat_rate(bc) will calculate the heat rate for bc="convection", "adiabatic", "isothermal", "infinite"
        fin.temperature(bc,x) will calculate the temperature as a function of bc and the location x
        fin.equations(T_b_name,T_infty_name,T_L_name,k_name,h_name,P_name,Ac_name,L_name) writes all the equations for you
            you need to run fin.heat_rate first.
    """
    def __init__(self,T_b,T_infty,T_L,k,h,P,Ac,L):
        self.T_b = T_b
        self.T_infty = T_infty
        self.T_L = T_L
        theta_b = T_b-T_infty
        theta_L = T_L-T_infty
        self.theta_b = T_b-T_infty
        self.theta_L = T_L-T_infty
        self.k = k
        self.h = h
        self.P = P
        self.Ac = Ac
        self.L = L
        m = np.sqrt(self.h*self.P/(self.k*self.Ac))
        self.m = m
        M = np.sqrt(self.h*self.P*self.k*self.Ac)*self.theta_b
        self.M = M
    def heat_rate(self,bc):
        self.bc = bc
        if self.bc == "convection":
            self.q_f = self.M*(np.sinh(self.m*self.L) + (self.h/(self.m*self.k))*np.cosh(self.m*self.L))/\
                    (np.cosh(self.m*self.L) + (self.h/(self.m*self.k))*np.sinh(self.m*self.L))
        elif self.bc == "adiabatic":
            self.q_f = self.M*np.tanh(self.m*self.L)
        elif self.bc == "isothermal":
            self.q_f = self.M*np.cosh(self.m*self.L - self.theta_L/self.theta_b)/np.sinh(self.m*self.L)
        elif self.bc == 'infinite':
            self.q_f = self.M
        else:
            print("boundary condition is not properly defined")
            
    def temperature(self,bc,x):
        self.bc = bc
        if self.bc == "convection":
            self.theta_over_theta_b = (np.cosh(self.m*(self.L-x)) + (self.h/(self.m*self.k))*np.sinh(self.m*(self.L-x)))/\
                    (np.cosh(self.m*self.L) + (self.h/(self.m*self.k))*np.sinh(self.m*self.L))
        elif self.bc == "adiabatic":
            self.theta_over_theta_b = np.cosh(self.m*(self.L-x))/np.cosh(self.m*self.L)
        elif self.bc == "isothermal":
            self.theta_over_theta_b = ((self.theta_L/self.theta_b)*np.sinh(self.m*self.L)+np.sinh(self.m*self.L - x))\
                                        /np.sinh(self.m*self.L)
        elif self.bc == 'infinite':
            self.theta_over_theta_b = np.exp(-self.m*x)
        else:
            print("boundary condition is not properly defined")
        self.T_x = self.T_infty + self.theta_over_theta_b*self.theta_b
        
    def equations(self,T_b_name,T_infty_name,T_L_name,k_name,h_name,P_name,Ac_name,L_name):
        T_x_sym = sym.symbols(r"T(x)")
        T_b_sym = sym.symbols(T_b_name)
        T_infty_sym = sym.symbols(T_infty_name)
        T_L_sym = sym.symbols(T_L_name)
        theta_sym = sym.symbols(r"\theta(x)")
        eq_theta = sym.Eq(theta_sym,T_x_sym-T_infty_sym)
        theta_b_sym = sym.symbols(r"\theta_b")
        eq_theta_b = sym.Eq(theta_b_sym,T_b_sym-T_infty_sym)
        theta_L_sym = sym.symbols(r"{\theta}_L")
        eq_theta_L = sym.Eq(theta_L_sym,T_L_sym-T_infty_sym)
        k_sym = sym.symbols(k_name)
        h_sym = sym.symbols(h_name)
        P_sym = sym.symbols(P_name)
        Ac_sym = sym.symbols(Ac_name)
        L_sym = sym.symbols(L_name)
        m_sym = sym.symbols(r"m")
        M_sym = sym.symbols(r"M")
        x_sym = sym.symbols(r"x")
        eq_m2 = sym.Eq(m_sym**2,h_sym*P_sym/(k_sym*Ac_sym))
        eq_M = sym.Eq(M_sym,sym.sqrt(h_sym*P_sym*k_sym*Ac_sym)*theta_b_sym)
        q_f_sym = sym.symbols(r"q_f")
        if self.bc == 'convection':
            eq_q = sym.Eq(q_f_sym,M_sym*(sym.sinh(m_sym*L_sym) + (h_sym/(m_sym*k_sym))*sym.cosh(m_sym*L_sym))/\
                    (sym.cosh(m_sym*L_sym) + (h_sym/(m_sym*k_sym))*sym.sinh(m_sym*L_sym)))
            eq_temp = sym.Eq(theta_sym/theta_b_sym,(sym.cosh(m_sym*(L_sym-x_sym)) + (h_sym/(m_sym*k_sym))*sym.sinh(m_sym*(L_sym-x_sym)))/\
                    (sym.cosh(m_sym*L_sym) + (h_sym/(m_sym*k_sym))*sym.sinh(m_sym*L_sym)))
        elif self.bc == "adiabatic":
            eq_q = sym.Eq(q_f_sym,M_sym*sym.tanh(m_sym*L_sym))
            eq_temp = sym.Eq(theta_sym/theta_b_sym,sym.cosh(m_sym*(L_sym-x_sym))/sym.cosh(m_sym*L_sym))
        elif self.bc == "isothermal":
            eq_q = sym.Eq(q_f_sym,M_sym*sym.cosh(m_sym*L_sym - theta_L_sym/theta_b_sym)/sym.sinh(m_sym*L_sym))
            eq_temp = sym.Eq(theta_sym/theta_b_sym,((theta_L_sym/theta_b_sym)*sym.sinh(m_sym*L_sym)+sym.sinh(m_sym*L_sym - x_sym))\
                                        /sym.sinh(m_sym*L_sym))
        elif self.bc == 'infinite':
            eq_q = sym.Eq(q_f_sym,M_sym)
            eq_temp = sym.Eq(theta_sym/theta_b_sym,sym.exp(-m_sym*x_sym))
        else:
            print("boundary condition is not properly defined")
        return display(eq_theta,eq_theta_b,eq_theta_L,eq_m2,eq_M,eq_q,eq_temp)
            
            
        

Overwriting HT_conduction_extended_surfaces.py


In [16]:
import  HT_conduction_extended_surfaces as extsurf
h = 100.
D = 0.005
P = np.pi*D
k = 398.
Ac = np.pi*(D**2)/4
T_b = 100.
T_infty = 25.
T_L = 0.
L = 0.04
fin = extsurf.ExtSurfaces(T_b,T_infty,T_L,k,h,P,Ac,L)
fin.heat_rate('isothermal')
print(fin.q_f)
fin.equations("T_b","T_\infty","T_L","k","h","P","A_c","L")
fin.temperature('infinite',L/2.)
print(fin.theta_over_theta_b,fin.T_x)

19.9199747892


\theta(x) = T(x) - T_\infty

\theta_b = -T_\infty + T_b

{\theta}_L = T_L - T_\infty

 2    P⋅h 
m  = ─────
     A_c⋅k

               ___________
M = \theta_b⋅╲╱ A_c⋅P⋅h⋅k 

            ⎛      {\theta}_L⎞
      M⋅cosh⎜L⋅m - ──────────⎟
            ⎝       \theta_b ⎠
q_f = ────────────────────────
             sinh(L⋅m)        

                            {\theta}_L⋅sinh(L⋅m)
            sinh(L⋅m - x) + ────────────────────
\theta(x)                         \theta_b      
───────── = ────────────────────────────────────
 \theta_b                sinh(L⋅m)              

(0.75310359671741289, 81.482769753805968)


### Library of Nu correlations for external flow around a pipe

In [17]:
%%file HT_external_convection.py
""" Object name 1: FlatPlate
    Object name 2: CircularCylinder
    Object name 3: NoncircularCylinder
    Object name 4: BankofTubes
"""
from __future__ import division
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')


from IPython.display import display,Image, Latex
import numpy as np
import math
import scipy.constants as sc

import sympy as sym
#from sympy import *

class FlatPlate(object):
    """ Definition of boundary layer thickness, friction coefficient, Nusselt number (both local and average)
        as a function of the regime.
        import HT_external_convection.py as extconv
        
        bl =extconv.FlatPlate(regime,thermal_bc,U_infty,nu,alpha_f,L,xi,Re_xc)
        where 
        regime = 'laminar' or 'turbulent' or 'mixed', 
        thermal_bc = 'isothermal', 'heat flux', 'unheated starting length',
        U_infty is the free stream velocity,
        nu the fluid viscosity,
        alpha the fluid thermal diffusivity,
        L length of the plate
        xi unheated started length
        Re_xc critical Reynolds number for transition laminar to turbulence
        
        bl.local(x) calculates the local Re, Cf, Nu at x based on thermal_bc
        
        bl.average(x) calculates the average Cf, Nu over a length x from the leading edge
        
        """
    def __init__(self,regime,thermal_bc,U_infty,nu,alpha,L,xi,Re_xc):
        self.regime = regime
        self.thermal_bc = thermal_bc
        self.U_infty = U_infty
        self.nu = nu
        self.alpha = alpha
        self.Pr = self.nu/self.alpha
        self.L = L
        self.xi = xi
        self.Re_xc = Re_xc
        self.Re_L = self.L*self.U_infty/self.nu
        self.x_c = self.Re_xc*self.nu/self.U_infty
        if self.regime != "laminar" and self.regime and "turbulent" and self.regime != "mixed":
            print("Warning: regime is not properly defined")
        if self.thermal_bc != "isothermal" and self.thermal_bc != "heat flux" and self.thermal_bc != "unheated starting length":
            print("Warning: thermal boundary condition is not properly defined")
        if self.Re_L > self.Re_xc and self.regime == "laminar":
            print("Warning: The end plate Reynolds number is larger than the critical Reynolds number, consider 'mixed' regime instead")
    def local(self,x):
        self.x = x
        self.Re_x = self.U_infty*self.x/self.nu
        if x == 0.:
            self.delta_x = 0.
            self.delta_Tx = 0.
            self.C_fx = 0.
            self.Nu_x = 0.
        else:
            if self.regime == "laminar":
                self.delta_x = 5.0*self.x/np.sqrt(self.Re_x)
                self.C_fx = 0.664*np.power(self.Re_x,-1./2.)
                if self.thermal_bc == "isothermal":
                    self.Nu_x = 0.332*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "heat flux":
                    self.Nu_x = 0.453*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "unheated starting length":
                    Re_xi = self.xi*self.U_infty/self.nu 
                    self.Nu_x = 0.332*np.power(self.Re_xi,1./2.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,3./4.),1./3.)
            elif self.regime == "turbulent":
                self.delta_x = 0.37*self.x*np.power(self.Re_x,-1./5.)
                self.C_fx = 0.0592*np.power(self.Re_x,-1./5.)
                if self.thermal_bc == "isothermal":
                    self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "heat flux":
                    self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "unheated starting length":
                    Re_xi = self.xi*self.U_infty/self.nu
                    self.Nu_x = 0.0296*np.power(self.Re_xi,4./5.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,9./10.),1./9.)
            elif self.regime == "mixed":
                if self.x < self.x_c:
                    self.delta_x = 5.0*self.x/np.sqrt(self.Re_x)
                    self.C_fx = 0.664*np.power(self.Re_x,-1./2.)
                    if self.thermal_bc == "isothermal":
                        self.Nu_x = 0.332*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "heat flux":
                        self.Nu_x = 0.453*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "unheated starting length":
                        Re_xi = self.xi*self.U_infty/self.nu 
                        self.Nu_x = 0.332*np.power(self.Re_xi,1./2.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,3./4.),1./3.)
                else:
                    self.delta_x = 0.37*self.x*np.power(self.Re_x,-1./5.)
                    self.C_fx = 0.0592*np.power(self.Re_x,-1./5.)
                    if self.thermal_bc == "isothermal":
                        self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "heat flux":
                        self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "unheated starting length":
                        Re_xi = self.xi*self.U_infty/self.nu
                        self.Nu_x = 0.0296*np.power(self.Re_xi,4./5.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,9./10.),1./9.)
                        
            self.delta_Tx = self.delta_x*np.power(self.Pr,-1./3.)
    def average(self,x):
        self.x = x
        self.Re_x = self.U_infty*self.x/self.nu
        if x == 0.:
            print("The length cannot be zero")
        if self.regime == "laminar":
            self.C_fave = 1.328*np.power(self.Re_x,-1./2.)
            if self.thermal_bc == "isothermal" or self.thermal_bc == "heat flux":
                self.Nu_ave = 0.664*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
            elif self.thermal_bc == "unheated starting length":
                p = 2.
                self.Nu_ave = 0.664*np.power(self.Re_xi,1./2.)*np.power(self.Pr,1./3.)*\
                              x/(x-xi)*np.power(1.-np.power(xi/x,(p+1.)/(p+2.)),p/(p+1.))
        elif self.regime == "turbulent":
            self.C_fave = 0.074*np.power(self.Re_x,-1./5.)
            if self.thermal_bc == "isothermal" or self.thermal_bc == "heat flux":
                self.Nu_ave = 0.037*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
            elif self.thermal_bc == "unheated starting length":
                p = 8.
                self.Nu_ave = 0.664*np.power(self.Re_xi,1./2.)*np.power(self.Pr,1./3.)*\
                              x/(x-xi)*np.power(1.-np.power(xi/x,(p+1.)/(p+2.)),p/(p+1.))
        elif self.regime == "mixed":
            A = 0.037*np.power(self.Re_xc,4./5.)-0.664*np.power(self.Re_xc,1./2.)
            
            self.C_fave = 0.074*np.power(self.Re_x,-1./5.) - 2.*A/self.Re_x
            self.Nu_ave = (0.037*np.power(self.Re_x,4./5.) - A)*np.power(self.Pr,1./3.)
                
class CircularCylinder(object):
    """ Nusselt correlations for cylinders
    import HT_external_convection.py as extconv
        
    bluff_body =extconv.CircularCylinder(Re,Pr,Pr_s) where Re, Pr, and Pr_s are the Reynolds number, Prandtl number
    of the flow and surface Prandtl number, respectively. If using Hilpert of Churchill Bernstein correlations,
    Re and Pr must be defined at film temperature, Pr_s can be set to anything since it is not used. 
    If using Zukauskas, Re and Pr are defined at temperature at infinity.
    
    bluff_body.correlation('Name of the correlation')
    Name of the correlation may be 'Hilpert', 'Churchill-Bernstein', 'Zukauskas'
    
    
    """
    def __init__(self,Re,Pr,Pr_s):
        self.Re = Re
        self.Pr = Pr
        self.Pr_s = Pr_s
    def correlation(self,name):
        self.name = name
        if self.name == "Hilpert":
            if self.Re < 0.4:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = 0.
            elif  self.Re < 4.:
                C = 0.989
                m = 0.33
            elif  self.Re < 40:
                C = 0.911
                m = 0.385
            elif self.Re < 4000:
                C = 0.683
                m = 0.466
            elif self.Re < 40000.:
                C = 0.193
                m = 0.618
            elif self.Re <= 400000.:
                C = 0.027
                m = 0.805
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = 0.
            if self.Re >= 0.4 and self.Re <= 400000.:
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
        elif self.name == "Churchill-Bernstein":
            if (self.Re*self.Pr < 0.2):
                print("Warning: Product RePr lower than acceptable limit for Churchill Bernstein Correlation")
                self.Nu = 0.
    
            else:
                self.Nu = 0.3+(0.62*self.Re**(0.5)*self.Pr**(1./3.)) \
                  /(1.+(0.4/self.Pr)**(2./3.))**(1./4.) \
                *(1.+(self.Re/282000.)**(5./8.))**(4./5.)
        elif self.name == "Zukauskas":
            if (self.Pr <= 10):
                n = 0.37
            else:
                n = 0.36
            if (self.Re < 1.) and (self.Re > 1.e6):
                print("Warning Reynolds number out of bounds for the Zukauskas Correlation")
                self.Nu = 0.
            else:
                if (self.Re < 40.):
                    C = 0.75
                    m = 0.4
                elif (self.Re < 1000.):
                    C = 0.51
                    m = 0.5
                elif (self.Re < 2.e5):
                    C = 0.26
                    m = 0.6
                else:
                    C = 0.076
                    m = 0.7
                self.Nu = C*self.Re**m*self.Pr**n*(self.Pr/self.Pr_s)**(1./4.)

class NonCircularCylinder(object):
    """ Nusselt correlations for  cylinders with non circular cross-sections.
    import HT_external_convection.py as extconv
        
    bluff_body =extconv.NonCircularCylinder(geometry,Re,Pr) where 
    geometry = "angled square" square with stagnation point on one of its edges
               "square" square with stagnation point at the center of one of its faces
               "angled hexagon" hexagon with stagnation point on one of its edges
               "hexagon" hexagon with stagnation point at the center of one of its faces
               "thin plate" thin plate perpendicular to the flow
    Re: Reynolds number at film temperature
    Pr: Prandtl number at film temperature
    
    Output: bluff_body.Nu, bluff_body.Nu_front, bluff_body.Nu_back, the last two are for thin plate only
    also bluff_body.geometry, bluff_body.Re, bluff_body.Pr
    
    """
    def __init__(self,geometry,Re,Pr):
        self.geometry = geometry
        self.Re = Re
        self.Pr = Pr
        if self.geometry == "angled square":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 6000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 60000.:
                C = 0.304
                m = 0.59
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "square":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 5000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 60000.:
                C = 0.158
                m = 0.66
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "angled hexagon":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 4500:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 90700.:
                C = 0.150
                m = 0.638
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "hexagon":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 5200:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 20400.:
                C = 0.164
                m = 0.638
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            elif  self.Re <= 105000.:
                C = 0.039
                m = 0.78
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "thin plate":
            self.Nu = np.inf
            if self.Re < 10000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu_front = np.inf
            elif  self.Re <= 50000.:
                C = 0.667
                m = 0.5
                self.Nu_front = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation for Nu_front")
                self.Nu_back = np.inf
            if self.Re < 7000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu_back = np.inf
            elif  self.Re <= 80000.:
                C = 0.191
                m = 0.667
                self.Nu_back = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation for Nu_front")
                self.Nu_back = np.inf
                
class BankofTubes(object):
    """ Nusselt correlations for flow across banks of tubes
    import HT_external_convection.py as extconv
        
    bank =extconv.BankofTubes(arrangement,V_i,D,nu,Pr,Pr_s,S_L,S_T,N_L,N_T) where 
    arrangement = "aligned" tubes are aligned in row and column
                  "staggered" tubes are staggered from one row to the next
    V_i: Inlet velocity
    Pr: Prandtl number at arithmetic mean temperature
    Pr_s: Prandtl number at surface temperature
    S_L: tube center to tube center separation  between two consecutive rows (perpendicular to the flow)
    S_T: tube center to tube center separation  between two consecutive rows (aligned with the flow)
    N_L: number of rows perpendicular to flow
    N_T: number of rows aligned with flow if unknown giev your best guess. 
    
    Output: bank.Nu: average Nusselt number
            bank.arrangement,.Re,.Pr,.Pr_s,.S_L,.S_T,.N_L,.N_T,.N=self.N_L*self.N_T
            
    bank.heat_rate(hbar,D,DT_lm) returns the heart per tube length based on the average convection coefficient
    hbar = bank.Nu*k/D, the tube diameter and the log mean temperature obtained from the function
    Delta_T_lm(T_s,T_i,T_o) where T_s is the surface temperature, T_i is the inlet temperature, T_o the outlet
    The outlet temperature is calculated from the function
    temperature_outlet_tube_banks(T_s,T_i,D,N,N_T,hbar,rho,V_i,S_T,Cp)
    
    self.Vmax
    """
    def __init__(self,arrangement,V_i,D,nu,Pr,Pr_s,S_L,S_T,N_L,N_T):
        self.arrangement = arrangement
        self.Pr = Pr
        self.Pr_s = Pr_s
        self.S_L = S_L
        self.S_T = S_T
        self.N_L = N_L
        self.N_T = N_T
        self.N = N_L*N_T
        self.D = D
        if self.arrangement == 'aligned':
            self.Vmax = self.S_T*V_i/(self.S_T-D)
        elif self.arrangement == 'staggered':
            self.S_D = np.sqrt(self.S_L**2+(self.S_T/2.)**2)
            self.Vmax = self.S_T*V_i/(2.*(self.S_D-D))
        Re = self.Vmax*self.D/nu
        self.Re = Re
        self.Nu = np.inf
        Corr_aligned = np.array([0.70,0.80,0.86,0.90,0.92,0.94,0.95,0.96,0.96,0.97,0.97,0.97,0.98,0.99,0.99,0.99,0.99,0.99,0.99])
        Corr_staggered = np.array([0.64,0.76,0.84,0.89,0.92,0.94,0.95,0.96,0.96,0.97,0.97,0.97,0.98,0.99,0.99,0.99,0.99,0.99,0.99])
        if (N_L < 20):
            if arrangement == 'aligned':
                Corr = Corr_aligned[N_L-1]
            elif arrangement == 'staggered':
                Corr = Corr_staggered[N_L-1]
        else:
            Corr = 1.
        if (Re < 10.):
            print('Warning: Re is out of bounds')
        if (Re >= 10.) and (Re <= 100.):
            if arrangement == 'aligned':
                C = 0.8
                m = 0.4
            elif arrangement == 'staggered':
                C = 0.9
                m = 0.4
            self.Nu = Corr*C*Re**m*Pr**(0.36)*(Pr/Pr_s)**(1./4.)
        elif (Re > 100.) and (Re <= 1000.):
            C = 0.51
            m = 0.
            self.Nu = Corr*C*Re**m*Pr**(0.36)*(Pr/Pr_s)**(1./4.)
        elif (Re > 1000.) and (Re <= 2.e5):
            if arrangement == 'aligned':
                if (S_T/S_L > 0.7):
                    C = 0.27
                    m = 0.63
                else:
                    print('Warning: inefficient, S_T/S_L<0.7')
                
            elif arrangement == 'staggered':
                if (S_T/S_L < 2):
                    C = 0.35*(S_T/S_L)**(1./5.)
                    m = 0.6
                else:
                    C = 0.40
                    m = 0.6
            self.Nu = Corr*C*Re**m*Pr**(0.36)*(Pr/Pr_s)**(1./4.)
        elif (Re > 2e5) and (Re <= 2.e6):
            if arrangement == 'aligned':
                C = 0.021
                m = 0.84
            elif arrangement == 'staggered':
                C = 0.022
                m = 0.84
            self.Nu = Corr*C*Re**m*Pr**(0.36)*(Pr/Pr_s)**(1./4.)
        else:
            print('Warning: Re is out of bounds')


    def heat_rate(self,hbar,D,DT_lm):
        self.q=self.N*hbar*np.pi*D*DT_lm

    def Vmax(self,V_i,D):
        if self.arrangement == 'aligned':
            self.Vmax = self.S_T*V_i/(self.S_T-D)
        elif self.arrangement == 'staggered':
            self.S_D = np.sqrt(self.S_L**2+(self.S_T/2.)**2)
            self.Vmax = self.S_T*V_i/(2.*(self.S_D-D))
            
def temperature_outlet_tube_banks(T_s,T_i,D,N,N_T,hbar,rho,V_i,S_T,Cp):
    return T_s-(T_s-T_i)*np.exp(-np.pi*D*N*hbar/(rho*V_i*N_T*S_T*Cp))

def N_L_for_given_To(T_s,T_i,T_o,D,hbar,rho,V_i,S_T,Cp):
    return np.log((T_s-T_i)/(T_s-T_o))/(np.pi*D*hbar)*(rho*V_i*S_T*Cp)

def Delta_T_lm(T_s,T_i,T_o):
    return ((T_s-T_i)-(T_s-T_o))/np.log((T_s-T_i)/(T_s-T_o))




Overwriting HT_external_convection.py


In [18]:
from scipy.constants.constants import C2K
from scipy.constants.constants import K2C
from scipy.constants.constants import F2K
from scipy.constants.constants import K2F
from scipy.constants.constants import C2F
from scipy.constants.constants import F2C

import HT_external_convection as extconv

square = extconv.NonCircularCylinder("thin plate",30000.,0.7)
print(square.Nu_back)

164.300640885


In [22]:
from scipy.constants.constants import C2K
from scipy.constants.constants import K2C
from scipy.constants.constants import F2K
from scipy.constants.constants import K2F
from scipy.constants.constants import C2F
from scipy.constants.constants import F2C


D = 10.e-3
S_T = 15.e-3
S_L = S_T
N_L = np.sqrt(196)
N_T = np.sqrt(196)
N = 196
T_i = 25.
V_i = 5.
T_s = 100.
T_o = T_s
T_m = (T_i+T_o)/2.
import thermodynamics as thermo
air = thermo.Fluid("air")
air.get_properties(C2K(T_m))
air_s = thermo.Fluid("air")
air_s.get_properties(C2K(T_s))

import HT_external_convection as extconv

?extconv.BankofTubes

nu = air.nu
Pr = air.Pr
Pr_s = air_s.Pr

bank = extconv.BankofTubes("aligned",V_i,D,nu,Pr,Pr_s,S_L,S_T,N_L,N_T)
print(bank.Re)
print(bank.Nu)
h = bank.Nu*air.k/D
T_o = extconv.temperature_outlet_tube_banks(T_s,T_i,D,N,N_T,h,air.rho,V_i,S_T,air.Cp)
DT_lm = extconv.Delta_T_lm(T_s,T_i,T_o)
bank.heat_rate(h,D,DT_lm)
print(bank.q)
print(T_o)

7830.85356304
67.4638616535
55034.7948454
74.0698511638


### Library of Nu correlations and functions for internal flow in pipes

In [ ]:
%%file HT_internal_convection.py
import numpy as np
import scipy
import scipy.optimize

def linear_interpolation(x_t,x_1,x_2,y_1,y_2):
    return y_1+(y_2-y_1)*(x_t-x_1)/(x_2-x_1)

def pressure_drop_pipe(f,L,D,rho,u_m):
    return f*(L/D)*(rho*u_m**2)/2.

def f_pipe_laminar(Re_D):
    return 64./Re_D

def f_pipe_colebrook(Re_D,eps):
    Re = Re_D
    e = eps
     
    f_0 = (0.790*np.log(Re)- 1.64)**(-2.)
    if (e > 0.):
        f_1 = 1./(-2.0*np.log10(e/3.71))**2
    else:
        f_1 = f_0
    f_guess = np.max(f_0,f_1)
    #f_guess = 0.04
    def f_tmp(x):
        y = (-2*np.log10((2.51/(Re*np.sqrt(x))) + (e/(3.71))) - 1.0/np.sqrt(x))
        return y
    y = scipy.optimize.fsolve(f_tmp, f_guess)
    return y
def log_mean_temperature(T_s,T_o,T_i):
    if (T_s < min(T_o,T_i)):
        DT_o = T_o-T_s
        DT_i = T_i-T_s
    elif (T_s > max(T_o,T_i)):
        DT_o = T_s-T_o
        DT_i = T_s-T_i
    return (DT_o-DT_i)/np.log(DT_o/DT_i)

def T_mx_Ts_constant(T_s,T_mi,P,mdot,Cp,hbar,x):
    return T_s-(T_s-T_mi)*np.exp(-P*x*hbar/(mdot*Cp))

def T_mo_T_infty(T_infty,T_mi,P,L,mdot,Cp,R_tot):
    return T_infty-(Tinfty-T_mi)*np.exp(-1/(mdot*Cp*Rtot))

def hbar_laminar_isothermal(k,D):
    return 3.66*k/D

def hbar_laminar_isoflux(k,D):
    return 4.36*k/D

def Nu_turbulent_Dittus_Boelter(Re,Pr,mode):
    if (mode == 'heating'):
        n = 0.4
    elif (mode == 'cooling'):
        n = 0.3
    return 0.023*Re**(4./5.)*Pr**n

def Nu_turbulent_Sieder_Tate(Re,Pr,mu,mu_s):
    return 0.027*Re**(4./5.)*Pr*(1./3.)*(mu/mu_s)**0.14

def Nu_turbulent_Gnielinski(Re,Pr,f):
    return (f/8.)*(Re-1000.)*Pr/(1+12.7*(f/8.)**0.5*(Pr**(2./3.)-1.))

def Nu_turbulent_Skupinski(Re,Pr):
    return 4.82+0.0185*(Re*Pr)**0.827

def Nu_turbulent_Seban(Re,Pr):
    return 5.0+0.025*(Re*Pr)**0.8

### Library for natural convection around cylinders

In [ ]:
%%file HT_natural_convection_cylinder.py
import numpy as np
import scipy
import scipy.optimize

def Gr(g,beta,DT,D,nu):
    return (g*beta*DT*D**3)/(nu**2)

def Ra(g,beta,DT,D,nu,alpha):
    return (g*beta*DT*D**3)/(nu*alpha)


def Nu_Morgan(Ra):
    if (Ra <= 1e-2):
        C=0.675
        n=0.058
    elif (Ra <= 1e2):
        C=1.02
        n=0.148
    elif (Ra <= 1e4):
        C=0.85
        n=0.188
    elif (Ra <= 1e7):
        C=0.480
        n=0.250
    elif (Ra <= 1e12):
        C=0.125
        n=0.333
    return C*Ra**n

def Nu_Churchill_Chu(Ra,Pr):
    return (0.60+(0.387*Ra**(1./6.))/(1.+(0.559/Pr)**(9./16.))**(8./27.))**2 


### Library of natural convection in enclosure

In [ ]:
%%file HT_natural_convection_enclosure.py
import numpy as np
import scipy
import scipy.optimize

def Gr(g,beta,DT,L,nu):
    return (g*beta*DT*L**3)/(nu**2)

def Ra(g,beta,DT,L,nu,alpha):
    return (g*beta*DT*L**3)/(nu*alpha)

def Nu_vertical_enclosure(Ra,Pr,H,L):
    if (H/L) < 2.:
        if Ra*Pr/(0.2+Pr)> 1.e3:
            Nu = 0.18*(Pr/(0.2+Pr)*Ra)**0.29
        else:
            print('Ra is too low for this correlation')
            Nu = np.inf
    elif H/L < 10:
        if Ra < 1e10:
            Nu = 0.22*(Pr/(0.2+Pr)*Ra)**0.28*(H/L)**(-0.25)
        else:
            print('Ra is too high for this correlation')
            Nu = np.inf
    elif Ra < 1e4:
        print('Ra is too low for this correlation')
        Nu = np.inf
    elif Ra < 1e7:
        if Pr > 0.6 and Pr < 2e4:
            print('ok')
            Nu =0.42*Ra**0.25*Pr**0.012*(H/L)**(-0.3)
        else :
            print('Pr is out of bounds for this correlation')
            Nu = np.inf
    elif Ra < 1e9:
        if Pr > 0.6 and Pr < 20.:
            Nu =0.46*Ra**(1./3.)
        else :
            print('Pr is out of bounds for this correlation')
            Nu = np.inf
    else:
        print('Ra is too high, got nothing for you')
        Nu = np.inf
    return Nu
            